In [ ]:
M3 = 20.115 / 24
M4 = 30.096 / 32
M5 = 40.092 / 42
M6 = 50.077 / 51
print(f"{(M3+M4+M5+M6)/4}")
M3, M4, M5, M6

In [8]:
import cv2
import numpy as np


def preprocess_image(image_path):
    # 画像の読み込み
    image = cv2.imread(image_path)
    end_x = -1
    start_x = 400
    end_y = 2048
    start_y = 0
    image = image[start_y:end_y, start_x:end_x]
    # グレースケール変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 画像の平滑化
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imwrite("result_gray.png", blurred)
    return blurred,image


def detect_circles(image_path, min_radius=10, max_radius=100, param1=50, param2=30, Tolerance=1.7, realvalue=0.05):
    # 画像の前処理
    preprocessed_image,image = preprocess_image(image_path)

    # ハフ変換を用いて円を検出
    circles = cv2.HoughCircles(
        preprocessed_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=20,
        param1=param1,
        param2=param2,
        minRadius=min_radius,
        maxRadius=max_radius
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 検出された円を描画
        result_image = cv2.imread(image_path)
        radiuss = []
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            radiuss.append(radius)

            cv2.circle(preprocessed_image, center,
                       radius, (0, 255, 0), 2)  # 円を描画
            # 合否結果判定
            Tolerancemin = Tolerance-0.05
            Tolerancemax = Tolerance+0.05
            # pxをmmに変換する
            realvalue_radius = radius*2*0.05  # 実際の寸法＝直径(半径×2)×1pxの長さ
            if realvalue_radius < Tolerancemax and realvalue_radius > Tolerancemin:
                gouhi = "o"
            else:
                gouhi = "o"

            x = round((3513-center[0])*0.05, 2)
            y = round((37-center[1])*0.05, 2)
            # 座標と直径を表示
            realvalue_radius = round(realvalue_radius, 2)
            text1 = f"x{x}mm"
            text4 = f"y{y}mm"
            text3 = f"center{Tolerance}+-0.05mm"
            text2 = f"center{realvalue_radius}mm, {gouhi}"

            cv2.putText(image, text1, (i[0] - radius, i[1] -
                        4*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, text4, (i[0] - radius, i[1] -
                        3*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, text2, (i[0] - radius, i[1] -
                        radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, text3, (i[0] - radius, i[1] -
                        2*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imwrite('/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/result_image.png', image)
        # cv2.imshow('Detected Circles', result_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindow+s()
    else:
        print("円が検出されませんでした。")


# 使用例
detect_circles('/home/kuga/img/pp1.png', min_radius=10,
               max_radius=100, param1=50, param2=30, realvalue=0.05)

In [ ]:
import cv2
import numpy as np

BINARY_THRESHOLD = 180
def preprocess_image(image_path):
    # 画像の読み込み
    image = cv2.imread(image_path)
    end_x = -1
    start_x = 400
    end_y = 2048
    start_y = 0
    image = image[start_y:end_y, start_x:end_x]
    # グレースケール変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray, BINARY_THRESHOLD, 255, cv2.THRESH_BINARY)

    # 画像の平滑化
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imwrite("result_gray.png", blurred)
    return binary_image


def detect_circles(image_data, min_radius=10, max_radius=100, param1=200, param2=15):
    # ハフ変換を用いて円を検出
    circles = cv2.HoughCircles(
        image_data,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=100,
        param1=param1,
        param2=param2,
        minRadius=min_radius,
        maxRadius=max_radius
    )
    result_image = cv2.cvtColor(image_data,cv2.COLOR_GRAY2BGR)
    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 検出された円を描画
        radiuss = []
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            radiuss.append(radius)
            cv2.circle(result_image, center,
                       radius, (0, 255, 0), 2)  # 円を描画
            
        cv2.imwrite("./detectCircle_result.png",result_image)
        # 新しい高さと幅を計算
        # new_height = height // 3
        # new_width = width // 3
        # resized_img = cv2.resize(result_image, (new_width, new_height))
        # cv2.imshow("sd", resized_img)
        # cv2.waitKey(-1)
        # cv2.destroyAllWindows()
    else:
        cv2.imwrite("./detectCircle_result.png",result_image)
        print("円が検出されませんでした。")


img= preprocess_image("./sample_images/accuracy_100_qr.png")
# 画像の高さと幅を取得
height, width = img.shape[:2]
detect_circles(img)

# 画像をリサイズ
# cv2.imshow("sd",resized_img)
# cv2.waitKey(-1)
# cv2.destroyAllWindows()

In [37]:
import cv2
import numpy as np


def preprocess_image(image_path):
    # 画像の読み込み
    image = cv2.imread(image_path)
    end_x = -1
    start_x = 400
    end_y = 2048
    start_y = 0
    image = image[start_y:end_y, start_x:end_x]
    # グレースケール変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # 2値化する
    _, gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # 画像の平滑化
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imwrite("result_gray.png", blurred)
    return blurred,image


def detect_circles(image_path, min_radius=10, max_radius=100, param1=50, param2=30, Tolerance=1.7, realvalue=0.05):
    # 画像の前処理
    preprocessed_image,image = preprocess_image(image_path)
    cv2.imwrite('/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/result_image2.png', preprocessed_image)
    # ハフ変換を用いて円を検出
    circles = cv2.HoughCircles(
        preprocessed_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=20,
        param1=param1,
        param2=param2,
        minRadius=min_radius,
        maxRadius=max_radius
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 検出された円を描画
        result_image = cv2.imread(image_path)
        radiuss = []
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            radiuss.append(radius)
            print(center,radius)
            cv2.circle(image, center,
                       radius, (0, 255, 0), 2)  # 円を描画
            # 合否結果判定
            Tolerancemin = Tolerance-0.05
            Tolerancemax = Tolerance+0.05
            # pxをmmに変換する
            realvalue_radius = radius*2*0.05  # 実際の寸法＝直径(半径×2)×1pxの長さ
            if realvalue_radius < Tolerancemax and realvalue_radius > Tolerancemin:
                gouhi = "o"
            else:
                gouhi = "o"

            x = round((3513-center[0])*0.05, 2)
            y = round((37-center[1])*0.05, 2)
            # 座標と直径を表示
            realvalue_radius = round(realvalue_radius, 2)
            text1 = f"x{x}mm"
            text4 = f"y{y}mm"
            text3 = f"center{Tolerance}+-0.1mm"
            text2 = f"center{realvalue_radius}mm, {gouhi}"
            if x == 90.35:
                cv2.putText(image, text1, (-200+i[0] - radius, i[1] -
                            5*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text4, (-200+i[0] - radius, i[1] -
                            4*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text2, (-200+i[0] - radius, i[1] -
                            2*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text3, (-200+i[0] - radius, i[1] -
                            3*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif y == -19.15:
                cv2.putText(image, text1, (-250+i[0] - radius, i[1] -
                            5*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text4, (-250+i[0] - radius, i[1] -
                            4*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text2, (-250+i[0] - radius, i[1] -
                            2*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text3, (-250+i[0] - radius, i[1] -
                            3*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(image, text1, (i[0] - radius, i[1] -
                            5*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text4, (i[0] - radius, i[1] -
                            4*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text2, (i[0] - radius, i[1] -
                            2*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(image, text3, (i[0] - radius, i[1] -
                            3*radius), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imwrite('/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/result_image.png', image)
        # cv2.imshow('Detected Circles', result_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindow+s()
    else:
        print("円が検出されませんでした。")


# 使用例
detect_circles('/home/kuga/img/pp1.png', min_radius=10,
               max_radius=100, param1=50, param2=30, realvalue=0.05)

(1706, 1628) 32
(1718, 420) 31
(1808, 326) 31
(1804, 1722) 31
(500, 1620) 24


In [3]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    # 画像の読み込み
    image = cv2.imread(image_path)
    end_x = -1
    start_x = 400
    end_y = 2048
    start_y = 0
    # image = image[start_y:end_y, start_x:end_x]
    # 2値化する
    _, image = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)
    # グレースケール変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 画像の平滑化
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imwrite("result_gray.png", blurred)
    return blurred,image

def save_cropped_circles(image_path, output_dir, min_radius=10, max_radius=100, param1=50, param2=30):
    # 画像の前処理
    preprocessed_image,result_image = preprocess_image(image_path)

    # ハフ変換を用いて円を検出
    circles = cv2.HoughCircles(
        preprocessed_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=20,
        param1=param1,
        param2=param2,
        minRadius=min_radius,
        maxRadius=max_radius
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 検出された円を描画

        for i, circle in enumerate(circles[0, :]):
            center = (circle[0], circle[1])
            radius = circle[2]

            # 円を描画
            cv2.circle(result_image, center, radius, (0, 255, 0), 2)

            # 円の部分を拡大して保存
            x, y, r = circle
            cropped = result_image[y-(2*r):y+(2*r), x-(2*r):x+(2*r)]
            print(x,y,r)
                # 円の中心にテキストを追加
            cv2.putText(result_image, f'{i}Circle', (int(x - radius), int(y - radius - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imwrite(os.path.join(output_dir, f'circle_{i}.png'), cropped)

        # 円を描画した画像を保存
        cv2.imwrite(os.path.join(output_dir, 'result.png'), result_image)
    else:
        print("円が検出されませんでした。")
        # 円を描画した画像を保存
        cv2.imwrite(os.path.join(output_dir, 'result.png'), result_image)

# 使用例
save_cropped_circles('/home/kuga/img/AQR/4/ACCURACY_ORIGINAL.png', '/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/sample_images', min_radius=10, max_radius=100, param1=50, param2=30)


920 1206 40
1320 1010 50
1318 1208 41
918 1404 30
1316 1608 24
920 1002 46
1320 1408 31
916 1604 22


1894 468 33
1728 202 33
1944 252 33
1680 420 34

In [5]:
import cv2
import numpy as np

def calculate_diameter(image_path):
    # 画像の読み込み
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"画像が読み込めませんでした: {image_path}")
        return None

    # 二値化
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # 輪郭抽出
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 最大の輪郭を見つける
    max_contour = max(contours, key=cv2.contourArea)

    # 最小包囲円を計算
    (x, y), radius = cv2.minEnclosingCircle(max_contour)

    # 直径を計算（ピクセル単位）
    diameter = 2 * radius
    # 円と直径を描画
    
    # cv2.circle(binary_image, (int(x), int(y)), int(radius), 127, 2)
    # cv2.line(binary_image, (int(x - radius), int(y)), (int(x + radius), int(y)), 125, 2)
    cv2.imwrite("/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/result_image.png",binary_image)
    return diameter

# 使用例
for i in range (8):
    diameter = calculate_diameter(f'/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/accuracy_inspection/sample_images/circle_{i}.png')
    print(f"円の直径（ピクセル単位）: {diameter}")
    # pxをmmに変換する
    realvalue_radius = diameter*0.04823527246796781  # 実際の寸法＝直径(半径×2)×1pxの長さ
    print(f"直径は{realvalue_radius}mm")

円の直径（ピクセル単位）: 84.13256072998047
直径は4.058156990238457mm
円の直径（ピクセル単位）: 103.28192901611328
直径は4.981831987109535mm
円の直径（ピクセル単位）: 85.58665466308594
直径は4.128295607295819mm
円の直径（ピクセル単位）: 64.63435363769531
直径は3.1176556585052198mm
円の直径（ピクセル単位）: 51.26422119140625
直径は2.47274367702565mm
円の直径（ピクセル単位）: 99.45408630371094
直径は4.797194950912283mm
円の直径（ピクセル単位）: 67.48250579833984
直径は3.25503705400414mm
円の直径（ピクセル単位）: 49.51093673706055
直径は2.3881735236564325mm
